In [1]:
# !pip install pyyaml pymongo 'pymongo[srv]'
# !python -m pip install 'mongo[srv]' dnspython


In [1]:
import pymongo, yaml
from src.LIB.UTILS.logs import get_logger
from src.linkedin_job_scraper import LinkedInJobScraper
from src.LIB.CONFIG.credentials import credentials

# get config file
with open("src/LIB/CONFIG/scraper.yaml", "r") as f:
    config = yaml.load(f)
    
# initialize logs
log = get_logger('LinkedInJobScraper') 

In [3]:
# connecting to database 
try:
    log.info('Trying to connect to database')
    client = pymongo.MongoClient(config['mongo_connect_url'].format(credentials['mongo_username'],credentials['mongo_password'],'jobs'))#,config['mongo_db'])
    db = client["linkedin_jobs"]
    collection = db["jobs"]
except Exception as e:
    log.error('Error connecting to database: ',e)


scraper = LinkedInJobScraper(100, None)
job_ids = scraper.search_jobs_ids('Software Engineer')
# get uniqe ids
job_ids = list(set(job_ids))
# filter ids that are not present in database
job_ids = list(filter(lambda x: collection.count({ '_id': x }, limit = 1) == 0, job_ids))
# get job descriptions
job_descriptions = [scraper.get_job_description(job_id) for job_id in job_ids]
# inserting jobs in database
for job in job_descriptions:    
    collection.insert_one(job)   

2021-03-04 14:47:08,150 — LinkedInJobScraper — INFO — <module>:3 — Trying to connect to database
2021-03-04 14:47:08,150 — LinkedInJobScraper — INFO — <module>:3 — Trying to connect to database
2021-03-04 14:47:08,251 — LinkedInJobScraper — INFO — search_jobs_ids:26 — Searching jobs in page 1/50
2021-03-04 14:47:08,251 — LinkedInJobScraper — INFO — search_jobs_ids:26 — Searching jobs in page 1/50
2021-03-04 14:47:09,159 — LinkedInJobScraper — INFO — search_jobs_ids:33 — Extracting Job Ids from the page
2021-03-04 14:47:09,159 — LinkedInJobScraper — INFO — search_jobs_ids:33 — Extracting Job Ids from the page
2021-03-04 14:47:09,164 — LinkedInJobScraper — INFO — search_jobs_ids:26 — Searching jobs in page 2/50
2021-03-04 14:47:09,164 — LinkedInJobScraper — INFO — search_jobs_ids:26 — Searching jobs in page 2/50
2021-03-04 14:47:09,960 — LinkedInJobScraper — INFO — search_jobs_ids:33 — Extracting Job Ids from the page
2021-03-04 14:47:09,960 — LinkedInJobScraper — INFO — search_jobs_ids: